In [223]:
import warnings
warnings.filterwarnings("ignore")
from data import *
from prep import *
from utils import *

### 데이터 개수
+ total 52,561
  + train 36,792  (70%)
  + pred 15,769   (30%)
  + normal 30,150
  + 비정상 22,411
  [ xss 8,070, beaconing 5,410, sql_injection 4,496, credential 4,435]

+ #### Model config load

In [2]:
model_config = get_config()

model name : [it_network_sl_model]
model config : {'common': {'model_name': 'it_network_sl_model', 'model_path': 'it_network_model_path', 'model_id': 337, 'config_name': 'it_network_supervised_learning', 'save_version': 'v1'}, 'train': {'optimizer': 'Adam', 'batch_size': 64, 'epochs': 100, 'learning_rate': 0.001}, 'predict': {'batch_size': 8, 'result_table': 'it_network_AD_result'}}


In [3]:
model_id = model_config['common']['model_id']
save_version = model_config['common']['save_version']
table = model_config['predict']['result_table']

+ ## CREATE TRAIN GRAPH

+ #### Data load(train & prediction)

In [4]:
dataClass = DataCreation(model_id)

NORMAL DATA START DATETIME:  2021-06-08 00:00:00
ATTACK DATA START DATETIME:  2021-06-08 00:00:00
DATA END DATETIME:  2021-06-10 23:59:59
******* NORMAL DATA LOAD *******
{'host': '192.168.0.42', 'port': '9001', 'timeout': '600000', 'thread': '10'}
******* NORMAL DATA INFO *******
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30150 entries, 0 to 30149
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype                     
---  ------  --------------  -----                     
 0   lgtime  30150 non-null  datetime64[ns, Asia/Seoul]
 1   src_ip  30150 non-null  object                    
 2   dst_ip  30150 non-null  object                    
 3   host    30150 non-null  object                    
 4   agent   30150 non-null  object                    
 5   query   30150 non-null  object                    
 6   label   30150 non-null  object                    
dtypes: datetime64[ns, Asia/Seoul](1), object(6)
memory usage: 1.6+ MB
None
******* ATTACK DATA LOAD *****

+ #### Train data

In [5]:
pd.concat([dataClass.train_str_data, dataClass.train_label_data], axis = 1)

,host,agent,query,label
0,www victim com,Mozilla Windows NT Win x Appl...,bbs modify php contents id <style> keyframe...,XSS
1,www victim com,Mozilla compatible MSIE Windows NT ...,bbs board php page writer JCK E,normal
2,www victim com,Mozilla Windows NT WOW AppleWebK...,bbs delete php contents id writer AE,normal
3,www victim com,Mozilla Windows NT Win x Appl...,bbs board php page writer g,normal
4,www victim com,Mozilla compatible MSIE Windows NT ...,bbs board php page writer O,normal
...,...,...,...,...
36787,www victim com,Mozilla compatible MSIE Windows NT ...,"bbs write php contents id ""nick PBX num xcelad...",CREDENTIAL
36788,www victim com,Mozilla CK {} Windows NT WOW Trid...,bbs write php contents id writer D,normal
36789,www victim com,Mozilla CK {} Windows NT WOW Trid...,bbs delete php contents id writer q,normal
36790,www victim com,Mozilla Windows NT Win x Appl...,bbs write php contents id writer YApg,normal


+ #### Preprocessing (one-hot encoding, TF-IDF)

+ #### Train data preprocessing transform & load from pickle

In [6]:
strPrep = StrProcessing()
train_prep_data = strPrep.load_tfidf_model_trans(dataClass.train_str_data, list(dataClass.train_str_data), 10000, save_version)

host model load complete **************
agent model load complete **************
query model load complete **************


In [7]:
X_train = train_prep_data.copy()
y_train = dataClass.train_label_data.copy()

+ #### label change

In [8]:
react_dict = {'XSS' : 'Validate and invalidate input and output values, Validation of length, character, format, and business rules for input data before data is entered'
        ,'CREDENTIAL' : 'Set the number of password failures, Introduction of additional authentication system, Big data analysis through web logging'
        ,'SQL_INJECTION' : 'Input validation, Using stored procedures, Allow access only to trusted networks and servers'
        ,'BEACONING' : 'Remove or disable extraneous applications, services, and daemons from your device, Block inbound and outbound traffic from suspicious endpoints at the network perimeter'}

In [9]:
for i in range(len(y_train)):
    if y_train.label[i] != 'normal':
        y_train.label[i] = react_dict[y_train.label[i]]

+ ### KeplerMapper

+ #### create normal & attack shuffle data

In [10]:
def create_shuffle_data(x, y, attack_name):
    total = pd.concat([x, y], axis = 1)
    normal_df = total[total.label == 'normal'].sample(n=50)
    attack_df = total[total.label == attack_name].sample(n=50)
    
    shuffle_df = pd.concat([normal_df, attack_df]).sample(frac=1)
    
    return shuffle_df

In [11]:
import sklearn
import kmapper as km
from itertools import product
import sys
import os

In [12]:
import stellargraph as sg
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import DeepGraphCNN
from stellargraph import StellarGraph
from stellargraph import datasets
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (1.8,1.8)

In [ ]:
mapper = km.KeplerMapper(verbose=0)
attack_list = list(set(y_train.label))
batch_size = 10
cnt = 1

for loof in range(800):
    for attack_name in attack_list:
        stellar_list = []

        if not os.path.exists(sys.path[0]+'/train_graph/'+ attack_name):
            os.makedirs(sys.path[0]+'/train_graph/'+ attack_name)
        
#         if not os.path.exists(sys.path[0]+'/train_image/'+ attack_name):
#             os.makedirs(sys.path[0]+'/train_image/'+ attack_name)
        
        save_raw_data = pd.DataFrame()
        
        for batch_length in range(batch_size):
            shuffle_df = create_shuffle_data(X_train, y_train, attack_name)
                        
            shuffle_df.drop('label', axis = 1, inplace = True)
            
            raw_data = pd.concat([dataClass.train_index_data.iloc[shuffle_df.index], y_train.iloc[shuffle_df.index]], axis = 1)
            save_raw_data = pd.concat([save_raw_data, raw_data])
            
            # 데이터 차원 축소 (고차원 데이터 -> 저차원)
            lens = mapper.fit_transform(shuffle_df, projection=sklearn.decomposition.PCA(n_components = 2))
            
          # 노드, 엣지, 메타 정보로 그래프 생성
            graph = mapper.map(
            lens,
            shuffle_df, #클러스터링을 실행할 원본 데이터
            clusterer=sklearn.cluster.KMeans(2),
            cover=km.Cover(n_cubes=10, perc_overlap=0.5) #렌즈 커버 구성표 / lens 데이터 기반 커버 구성 / 하이퍼큐브의 수와 간격 (중첩)을 조절, 0.5 = 50% 중첩
        )
            # 차원 축소된 lens 데이터를 기반으로 hypercube를 생성, hypercube별 인덱스를 추출, 추출된 인덱스를 원본데이터에서 조회, hypercube별(원본) cluster 진행
            
            
#             km.draw_matplotlib(graph)
#             plt.savefig(sys.path[0] + "/train_image/{}/".format(attack_name) + attack_name + '_' + str(cnt)+'.png')
            
            links_key = list(graph['links'].keys())
            
            product_list = []
            for link_key in links_key:
                if len(graph['links'][link_key]) != 0:
                    for m in range(len(graph['links'][link_key])):
                        try:
                            for k in product(graph['nodes'][link_key], graph['nodes'][graph['links'][link_key][m]], repeat=1):
                                product_list.append(list(k))
                        except:
                            pass
                        
            for node_key in graph['nodes'].keys():
                for k in product(graph['nodes'][node_key], graph['nodes'][node_key], repeat=1):
                    product_list.append(list(k))
                    
            product_df = pd.DataFrame(data=product_list, columns=['source', 'target'])
            product_df.drop_duplicates(inplace=True)
            
            stellar_graph = sg.StellarGraph(shuffle_df.values, product_df)
            stellar_list.append(stellar_graph)
            
#             km.draw_matplotlib(graph)
        
        with open(sys.path[0] + "/train_graph/{}/".format(attack_name) + attack_name + '_' + str(cnt) + ".pickle", "wb") as f:
            pickle.dump(stellar_list, f)

    cnt += 1

+ ## CREATE PREDICTION GRAPH

+ #### prediction data

In [19]:
pd.concat([dataClass.pred_str_data, dataClass.pred_label_data], axis = 1)

,host,agent,query,label
36792,www victim com,Mozilla Windows NT AppleWebKit ...,bbs board php page writer K,normal
36793,www victim com,Mozilla Windows NT Win x Apple...,bbs write php contents id writer CHgFx,normal
36794,www victim com,Mozilla compatible MSIE Windows NT ...,bbs write php contents id ' e e c ...,BEACONING
36795,www victim com,Mozilla compatible MSIE Windows NT ...,bbs board php page writer FK,normal
36796,www victim com,Mozilla Windows NT AppleWebKit ...,bbs modify php contents id writer lq,normal
...,...,...,...,...
52556,www victim com,Mozilla Windows NT Win x Appl...,bbs write php contents id ' pathsearcherdete...,BEACONING
52557,www victim com,Mozilla Windows NT Win x Appl...,bbs board php page writer VY tb,normal
52558,www victim com,Mozilla compatible MSIE Windows NT ...,bbs delete php contents id writer qQL,normal
52559,www victim com,Mozilla Windows NT Win x Appl...,bbs write php contents id 'username <N A> num ...,CREDENTIAL


+ #### Prediction data preprocessing transform & load from pickle

In [20]:
pred_prep_data = strPrep.load_tfidf_model_trans(dataClass.pred_str_data, list(dataClass.pred_str_data), 10000, save_version)

host model load complete **************
agent model load complete **************
query model load complete **************


In [21]:
X_pred = pred_prep_data.copy()
y_pred = dataClass.pred_label_data.copy()
pred_index_data = dataClass.pred_index_data.copy()

In [22]:
y_pred.reset_index(drop = True, inplace =True)
pred_index_data.reset_index(drop = True, inplace =True)

+ #### label change

In [23]:
for i in range(len(y_pred)):
    if y_pred.label[i] != 'normal':
        y_pred.label[i] = react_dict[y_pred.label[i]]

+ ### KeplerMapper

In [220]:
mapper = km.KeplerMapper(verbose=0)
attack_list = list(set(y_pred.label))
batch_size = 10
cnt = 1

for loof in range(100):
    for attack_name in attack_list:
        stellar_list = []

        if not os.path.exists(sys.path[0]+'/pred_graph/'+ attack_name):
            os.makedirs(sys.path[0]+'/pred_graph/'+ attack_name)
            
        save_raw_data = pd.DataFrame()
    
        for batch_length in range(batch_size):
            shuffle_df = create_shuffle_data(X_pred, y_pred, attack_name)
            shuffle_df.drop('label', axis = 1, inplace = True)
            
            raw_data = pd.concat([pred_index_data.iloc[shuffle_df.index], y_pred.iloc[shuffle_df.index]], axis = 1)
            
            lens = mapper.fit_transform(shuffle_df, projection=sklearn.decomposition.PCA(n_components = 2))
            
            # 노드, 엣지, 메타 정보로 그래프 생성
            graph = mapper.map(
            lens,
            shuffle_df,
            clusterer=sklearn.cluster.KMeans(2),
            cover=km.Cover(n_cubes=15, perc_overlap=0.5)
        )
            links_key = list(graph['links'].keys())

            product_list = []
            for link_key in links_key:
                if len(graph['links'][link_key]) != 0:
                    for m in range(len(graph['links'][link_key])):
                        try:
                            for k in product(graph['nodes'][link_key], graph['nodes'][graph['links'][link_key][m]], repeat=1):
                                product_list.append(list(k))
                        except:
                            pass
            
            for node_key in graph['nodes'].keys():
                for k in product(graph['nodes'][node_key], graph['nodes'][node_key], repeat=1):
                    product_list.append(list(k))
                        
            product_df = pd.DataFrame(data=product_list, columns=['source', 'target'])
            product_df.drop_duplicates(inplace=True)
            
            stellar_graph = sg.StellarGraph(shuffle_df.values, product_df)
            
            stellar_list.append([list(np.array(raw_data)), stellar_graph])

        with open(sys.path[0] + "/pred_graph/{}/".format(attack_name) + attack_name + '_' + str(cnt) + ".pickle", "wb") as f:
            pickle.dump(stellar_list, f)
            
    cnt += 1